In [45]:

import aicsimageio as aio
from omero.gateway import BlitzGateway
import os
import json
tmp_path = '/tmp/'
import time
tot = time.time()
import sys
import omero
sys.path.append('/home/staffan/Code_KR/utils/')
import xmltodict
import aicsimageio as aio 
#from aicsimageio import AICSImage, imread
import shapely as shp
from shapely.geometry import Polygon, Point, MultiPolygon
import pandas as pd
import pylab as plt
import numpy as np
import re

def get_pol_area(roi_points):

    polygon = Polygon(roi_points)

    return polygon.area

def get_ome_anno(img_id,host="localhost",show=False,user='john',pwd="11235813"):
    conn = BlitzGateway(user,pwd , host=host, port=4064)
    conn.connect()
    conn.c.enableKeepAlive(300)
    

    roi_service = conn.getRoiService()
    #print(roi_service)
    result = roi_service.findByImage(img_id, None)
    #print(result)

    roi_json = {}

    for roi in result.rois:
        print('1')
        for s in roi.copyShapes():
            shape = {}
            shape['id'] = s.getId().getValue()
            shape['theT'] = s.getTheT().getValue()
            shape['theZ'] = s.getTheZ().getValue()
            if s.getTextValue():
                s1=s.getTextValue().getValue().replace('/', '_')
                shape['textValue'] = s1
            if type(s) == omero.model.RectangleI:
                shape['type'] = 'Rectangle'
                shape['x'] = s.getX().getValue()
                shape['y'] = s.getY().getValue()
                shape['width'] = s.getWidth().getValue()
                shape['height'] = s.getHeight().getValue()
            elif type(s) == omero.model.EllipseI:
                shape['type'] = 'Ellipse'
                shape['x'] = s.getX().getValue()
                shape['y'] = s.getY().getValue()
                shape['radiusX'] = s.getRadiusX().getValue()
                shape['radiusY'] = s.getRadiusY().getValue()
            elif type(s) == omero.model.PointI:
                shape['type'] = 'Point'
                shape['x'] = s.getX().getValue()
                shape['y'] = s.getY().getValue()
            elif type(s) == omero.model.LineI:
                shape['type'] = 'Line'
                shape['x1'] = s.getX1().getValue()
                shape['x2'] = s.getX2().getValue()
                shape['y1'] = s.getY1().getValue()
                shape['y2'] = s.getY2().getValue()
            elif type(s) == omero.model.MaskI:
                shape['type'] = 'Mask'
                shape['x'] = s.getX().getValue()
                shape['y'] = s.getY().getValue()
                shape['width'] = s.getWidth().getValue()
                shape['height'] = s.getHeight().getValue()
            elif type(s) == omero.model.PolygonI:
                #print('polygon')
                shape['type'] = 'Polygon'
                shape['points'] =[[int(float(i.split(',')[0])),int(float(i.split(',')[1]))] for i in s.getPoints().getValue().split(' ')]
                shape['area']= get_pol_area(shape['points'])
                
                #KR added in area to use for normalisation further down the line
            
                #shape['points'] = s.getPoints().getValue()
            elif type(s) in (
                    omero.model.LabelI, ):
                print(type(s), " Not supported by this code")
            # Do some processing here, or just print:
            if show:
                print("   Shape:",)
                for key, value in shape.items():
                    print( "  ", key, value,)
                

            roi_json[roi.getId().getValue()] = shape
            


    conn.close()
    
    return roi_json

def get_image_path(imageId,host='localhost',user='john',pwd="11235813"):
    """
    return the path location
    """
    conn = BlitzGateway(user,pwd , host=host, port=4064)
    conn.connect()
    image = conn.getObject("Image", imageId)
    tpath = 'data/OMERO/ManagedRepository/'+image.getImportedImageFilePaths()['server_paths'][0]
    #print(image.getImportedImageFilePaths()['server_paths'])
    print(tpath)

    conn.close()
    # print(host,host=='192.168.193.15',tpath)
    if host in ['192.168.193.15','172.27.107.136','localhost']: 
        if os.path.exists('/'+ tpath):
            print('Found the images')
            return '/'+tpath

    print('copying the whole slide image to localserver, please remember to remove that after use')
    dst_path = '%s%s'%(tmp_path,tpath.split('/')[-1])
    if not os.path.exists(dst_path):
        tflag= os.system('scp john-imac:/%s %s'%(tpath,tmp_path))
    else:
        return dst_path
    if tflag ==0 and os.path.exists(dst_path):
        return dst_path
    else:
        return False
    
def get_meta_path(imageId, project, subproject, outputdir, host='localhost',user='john',pwd="11235813"):
    conn = BlitzGateway(user,pwd , host=host, port=4064)
    conn.connect()
    conn.c.enableKeepAlive(300)
    image = conn.getObject("Image", imageId)
    tpath = image.getImportedImageFilePaths()['server_paths'][0]
    tcpath = image.getImportedImageFilePaths()['client_paths'][0]
    dataset=image.getParent()
    datasetname=dataset.getName()
    print('dataset is:', datasetname, 'json is')
    ann = dataset.getAnnotation()
    ann_list=dataset.listAnnotations()
    
    project=project
    directory = os.path.join(outputdir,project,subproject,'Ref/')
    for folder in os.listdir(directory):
        if folder[:4]==datasetname:
            meta_path = directory+folder+'/Images/FieldTables.json'
            print('metapath found', meta_path)
            return meta_path
    
      

def get_cluster_id(target_size,filepath, json_file):
    """
    Get cluster id based on image size
    ## Doesn't work, need to based on image name as well
    """
    
    tsx,tsy = target_size
    
    ta = json.loads(open(json_file,'r').read())
    if filepath is not None:
        filename = filepath.split('/')[-1]
        
        
        tcluster = int(filename.split('_F')[1].split('P')[0])-1
        
            #print(filename,tcluster)
        
        if ta[0]['FieldTables'][tcluster]['Dims'][0]==tsy and ta[0]['FieldTables'][tcluster]['Dims'][1]==tsx:
            return tcluster
        elif ta[0]['FieldTables'][tcluster]['Dims'][0]==tsx and ta[0]['FieldTables'][tcluster]['Dims'][1]==tsy:
            
            return tcluster
        
    
    #print(len(ta[0]['FieldTables']))
    for tcluster  in range(len(ta[0]['FieldTables'])):
        #print(ta[0]['FieldTables'][tcluster]['Dims'])
        if ta[0]['FieldTables'][tcluster]['Dims'][0]==tsx and ta[0]['FieldTables'][tcluster]['Dims'][1]==tsy:
            return tcluster
    tsy,tsx = target_size
    ta = json.loads(open(json_file,'r').read())
    #print(len(ta[0]['FieldTables']))
    for tcluster  in range(len(ta[0]['FieldTables'])):
        #print(ta[0]['FieldTables'][tcluster]['Dims'])
        if ta[0]['FieldTables'][tcluster]['Dims'][0]==tsx and ta[0]['FieldTables'][tcluster]['Dims'][1]==tsy:
            return tcluster
    

def get_target_field(fcenters,tclusterid,json_file):
    """
    input = index of the fcenters
    return targeted field, offsetx, offsety in the tile, startx, starty in the wsi
    """
    cell_x,cell_y = fcenters
    #print('roi points:', cell_x,cell_y)
    
    ta = json.loads(open(json_file,'r').read())
    
    tfield_num = len(ta[0]['FieldTables'][tclusterid]['FieldTable']['Field'])
    #print('tfield_num is:', tfield_num)
    binning =ta[0]['FieldTables'][0]['Binning']
    for tindex in range(tfield_num):
        #loops all fields in the cluster (one field is 1987 pixels)
    
        #print(ta[0]['FieldTables'][tcluster]['FieldTable']['X'][tindex])
        tfx = ta[0]['FieldTables'][tclusterid]['FieldTable']['X'][tindex]

        tfy = ta[0]['FieldTables'][tclusterid]['FieldTable']['Y'][tindex]
        #print('tfx and tfy from json file:', tfx,tfy)
        if cell_x >tfx and cell_x < tfx+(2160//binning) and cell_y > tfy and cell_y <tfy+(2160//binning):
            #print('ROI within bounds of field given', ta[0]['FieldTables'][tclusterid]['FieldTable']['Field'][tindex])
            #if roi x,y is lower than field coords then it is not in field. 
            
                #return tindex
            #print('tfx and tfy from json file:', tfx,tfy)
                #ffa.append([tindex,tfx,tfy]) targetfieldId,txx,tyy,ofx,ofy
            #print('the output of get target field is:', 'field ID:', ta[0]['FieldTables'][tclusterid]['FieldTable']['Field'][tindex], 'txx:',
                     #cell_x-tfx, 'txy:', cell_y-tfy,'ofx', tfx,'ofy', tfy)

            return ta[0]['FieldTables'][tclusterid]['FieldTable']['Field'][tindex],\
                     cell_x-tfx, cell_y-tfy,tfx,tfy
    
        
            


def get_xml_ref(imageId, project, subproject, outputdir):
    conn = BlitzGateway(user, password , host=host, port=4064)
    conn.connect()
    image = conn.getObject("Image", imageId)
    tpath = image.getImportedImageFilePaths()['server_paths'][0]
    tcpath = image.getImportedImageFilePaths()['client_paths'][0]
    dataset=image.getParent()
    datasetname=dataset.getName()
    ann = dataset.getAnnotation()
    ann_list=dataset.listAnnotations()
    print(datasetname)
    
    
    directory = os.path.join(outputdir, project,subproject, 'Ref/')
    for folder in os.listdir(directory):
        if folder[:4]==datasetname:
            xml_path = directory+folder+'/Images/Index.ref.xml'
            #print('xml path found', xml_path)
            return xml_path 
    
def get_field_position_old(targetfield,parsed_xml):
    # The below means I don't need to find and replace tdict with parsed_xml when rewriting this function. 
    tdict = parsed_xml
    for i  in range(len(tdict['EvaluationInputData']['Wells']['Well']['Image'])):
        tff = int(tdict['EvaluationInputData']['Wells']['Well']['Image'][i]['@id'].split('P')[0].split('F')[1])
        if tff == targetfield and int(tdict['EvaluationInputData']['Images']['Image'][i]['FieldID'])==targetfield:
            #print('get field position uses xml data from harmony, goes through all images (fields, i think), turns image name into tff(', tff, ') if tff is same as target field(', targetfield, ') from json cluster field, then it returns obx:', float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionX']['#text']), ', oby:', float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionY']['#text']), ', and res:', float(tdict['EvaluationInputData']['Images']['Image'][i]['ImageResolutionX']['#text']))
            #print(i)
            return float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionX']['#text']),\
                    float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionY']['#text']),\
                    float(tdict['EvaluationInputData']['Images']['Image'][i]['ImageResolutionX']['#text'])
    return None 

def get_field_position(targetfield,ref_xml): ## 58s
    fd=open(ref_xml)
    tdict = xmltodict.parse(fd.read())
    tlist=[int(tdict['EvaluationInputData']['Wells']['Well']['Image'][i]['@id'].split('P')[0].split('F')[1]) \
                   for i in range(len(tdict['EvaluationInputData']['Wells']['Well']['Image']))]
    ti = tlist.index(targetfield)
    if int(tdict['EvaluationInputData']['Images']['Image'][ti]['FieldID'])==targetfield:
        #print(i)
        return float(tdict['EvaluationInputData']['Images']['Image'][ti]['PositionX']['#text']),\
                float(tdict['EvaluationInputData']['Images']['Image'][ti]['PositionY']['#text']),\
                float(tdict['EvaluationInputData']['Images']['Image'][ti]['ImageResolutionX']['#text'])


#     for i  in range(len(tdict['EvaluationInputData']['Wells']['Well']['Image'])):
#         tff = int(tdict['EvaluationInputData']['Wells']['Well']['Image'][i]['@id'].split('P')[0].split('F')[1])
#         if tff == targetfield and int(tdict['EvaluationInputData']['Images']['Image'][i]['FieldID'])==targetfield:
#             #print(i)
#             return float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionX']['#text']),\
#                     float(tdict['EvaluationInputData']['Images']['Image'][i]['PositionY']['#text']),\
#                     float(tdict['EvaluationInputData']['Images']['Image'][i]['ImageResolutionX']['#text'])
    return None

def convert_cords(tps,ref_xml,tclusterid, json_file):
    """
    input points [[tpx,tpy]]
    return the transformed cordinates
    """
    results = []
    
    for tp in tps:
        #a[0]['FieldTables'][tclusterid]['FieldTable']['Field'][tindex], cell_x-tfx, cell_y-tfy,tfx,tfy
        
        tpx,tpy = tp
        targetfieldId,txx,tyy,ofx,ofy = get_target_field([tpx, tpy],tclusterid,json_file)
        
        obx,oby,res = get_field_position_old(targetfieldId,ref_xml)
        results.append([(obx+res*txx-1080*res)*1000000, (oby-res*tyy+0*res+1080*res)*1000000])
        #print('the results from get target field and get field position are then put into a list containing obx(xml)+res(xml)*txx(omero_x-json_x)*1 million,,, and oby-res*tyy+0*res+1080*res*1000000. results list is:', results)
    return results

def segment_data(user, password,imagename, imageId,rawdata, csv_object_reference):


# Get meta data from OMERO¶
    print('Getting metadata from OMERO...')
    st = time.time()
    tanno = get_ome_anno(imageId,user=user,pwd=password)
    #print(imageId, tanno)
    
    dst = get_image_path(imageId,host='localhost',user=user,pwd=password)
    dst_json = get_meta_path(imageId,project, subproject, outputdir, host='localhost',user=user,pwd=password)
    print(time.time()-st)
    # print('metapath is:', dst_json, 'image path is:', dst)
    





    # Reading target image, determine pixel dimensions. Used to identify witch cluster.
    print('Reading images, identifying clusters (sections)...')
    timg = aio.imread(dst)
    # print(timg.shape)
    tclusterid= get_cluster_id(timg.shape[-2:], dst, dst_json)
    
    ref_xml = get_xml_ref(imageId, project, subproject, outputdir)
    print('ref xml is:', ref_xml)
    print(time.time()-st)
    print('...Done')


    # Load OMERO ROI coordinates - convert to Harmony scale
    print('Converting ROI coordinates...')
    st = time.time()

    dftanno = pd.DataFrame(tanno)
    #print('omero image shape data:', dftanno)
   
    sertanno =  dftanno.loc['points', :]
   
    ROI_li = sertanno.tolist()
    #print('tps is each object in Roi_li, this comes from omero shape ROI data', ROI_li)
   
    ROI_area=dftanno.loc['area']
    
    
    ROI_Area=ROI_area.tolist()


    # Load ROI names
    ROInames = dftanno.loc['textValue']
    
    ROI_names = ROInames.tolist()
   

    cROI_li = list()
    # This reads the XML file once, and then we use that in the loop
    # Rather than opening it a bazillion times. Should save a lot of time.
    fd=open(ref_xml)
    ref_xml = xmltodict.parse(fd.read())
    for i in ROI_li:
        #area_list.append(pol_area)
        # convert each set of coords.
        cROI = convert_cords(i,ref_xml,tclusterid, json_file=dst_json)
        
        cROI_li.append(cROI)
        

    print(time.time()-st)
    print('All Done!')




    # Segment data
    # Create Bounding box for combined polygons in one cluster. 
    st = time.time()
    pli = list()
    for i in range(len(cROI_li)):
        pli.append(Polygon((cROI_li[i])))
        #print(i)
    bbox = MultiPolygon(pli)
    bbox.bounds # Bounding box for mulipolygon
    # minx, miny, maxx, maxy 
    print(time.time()-st)

    # Load analysis results - filter away all cells outside the ROIs of this image.
    st = time.time()
    pf = pd.read_csv(rawdata, header=8, delimiter='\t')
    
    
    # pf # pandaframe
    # data loaded here is for entire slide!

    # This slices away data from other clusters.
    # How to Select Rows of Pandas Dataframe Based on a Single Value of a Column?
    pf.shape
    fpf = pf
    is_ymin = fpf['Position Y [µm]'] > bbox.bounds[1] # Y = 27852.946882244712
    fpf = fpf[is_ymin]
    is_ymax = fpf['Position Y [µm]'] < bbox.bounds[3] # Y = 28809.233162833487
    fpf = fpf[is_ymax]
    is_xmin = fpf['Position X [µm]'] > bbox.bounds[0] # X = 888.0532437902484
    fpf = fpf[is_xmin]
    is_xmax = fpf['Position X [µm]'] < bbox.bounds[2] # X = 2489.3144912603493
    fpf = fpf[is_xmax]
    
    

    
    # fpf # filtered pandaframe

    fpf['ROI'] = 'not_assigned'
    # Add column for ROI-attribute to the filtered dataframe.

    # Johns way of doing it in a one liner
    # fpff = pf.loc[(pf['Position Y [µm]']>bbox.bounds[1])&(pf['Position Y [µm]']<bbox.bounds[3])&(pf['Position X [µm]']>bbox.bounds[0])&(pf['Position X [µm]']<bbox.bounds[2])]
    # print(fpff.shape)

    # I could slice the data down to each polygons bbox. Might improve speed if .contains(coords) is slow.
    print(time.time()-st)


    # Segmentation.

    st = time.time()
    # fpf = filtered data (inside bounding box of the combined ROIs.)
    # pli = the multipolygon = a list of all individual ROIs.

    ##### Currently can only insert into the filtered data. But the file has maintained the original Index it seems.

    for i in range(len(fpf)) :     
        # print(fpf.iloc[i, 8], fpf.iloc[i, 9])
        x = fpf.iloc[i, 8]
        y = fpf.iloc[i, 9]
        coord = [x,y]
    #     print(coord)
        cord = Point(coord)
        
    #     print(bbox.contains(cord))
    #     print(pli[1].contains(cord))
    #     if pli[2].contains(cord)==True:
    #          print(coord)
        for a in range(len(pli)) :   
    #         print(a)
            if pli[a].contains(cord)==True:
    #             print(coord) # The coord
    #             print(ROI_names[a]) # The ROI name
    #             print(fpf.iloc[i,-1]) # The original index
                ROI = ROI_names[a]
                
    #             print(ROI)
    #             print(id)
    #             for index, row in pf_res.iterrows():
    #                  pf_res.at[index, 'ROI'] = ROI
                fpf.iloc[i, -1] = ROI
                
            
        

                # pf_res.at[pl, 'ROI'] = str(ROI)
    # df.at[index, 'new_column'] = new_value
    print(time.time()-st)

    # Results - Objects per ROI in separate CSV files

    st = time.time()
    # using list
    results = list()
    
    for r in ROI_names:
        #print('ROI IS', r, 'roi list is', ROI_names)
        
        results.append(fpf.loc[(fpf['ROI']==r)])
        #print(results)
        
        

    return (results, ROI_names, ROI_Area)

def handle_csv(results, ROI_names, imagename, csv_object_reference, ROI_Area, homepath, project, subproject):

        
    fp = homepath+project+'_'+subproject+'/In_Progress/'+str(imagename)+'_'+csv_object_reference+'_results_{}.csv'.format(i)
        
    r['Condition']=imagename.split('_')[2]
    if not 'S' in imagename[:4]:
        r['Section']= imagename.split('S')[1].split('_')[0]
    else:
        r['Section']= imagename.split('S')[2].split('_')[0]
    if '_' in csv_object_reference:
        r['Object']=csv_object_reference.split('_')[0]
    else:
        r['Object']=csv_object_reference.split('.')[0]
    r['Measurement']=imagename[:4]
    r['Region_Area']=a
  
        #print(r.head(5))
    
    return r, fp

def harmony_omero_output (user, password, host, datasets, directory):
    
    all_data = []
    pathlist=[]
    csv_list=[]
    joined_data=[]
    

    with BlitzGateway(user , password, host=host, port=4064) as conn:

            conn.connect()
            conn.c.enableKeepAlive(300) # 120-300? Certainly less than 600
            print('connected')
            #print(user, password, host, datasets, directory)

            for dataset in datasets:
                for image in conn.getObjects('Image', opts={'dataset': dataset}):
                    #print(image)
                    imageId = image.getid()
                    imagename=image.getName()
                    dataset=image.getParent()
                    dataseto=dataset.getName()
                    datasetname=dataseto.upper()
                    this_iteration = {'imageId': imageId, 'imagename': imagename, 'dataset': dataset, 'datasetname': datasetname}
                    all_data.append(this_iteration)
                    #print (all_data)
                   
                    
                    #print('all data length is:', len(all_data))
                    #all data is a list of dictionaries, 8 dictionaries, each corresponding to one image, 8 images in total.
                    #print (imageId, imagename, datasetname, dataset.getid())
                    
    ## NEED TO FIND AN EFFICIENT WAY TO CHOOSE WHICH MEASUREMENTS FROM THE DIRECTORY TO SEGMENT

    for filename in os.listdir(directory):
        #print(filename)
        if 'Measurement' in filename:
            # loops through the two measurements x2
            for file in os.listdir(directory+filename):
                regexmatch = re.match('[a-zA-Z]*\d', file)
                if regexmatch:
                    print('regex match', regexmatch[0])
                    csv_directory=directory+filename+'/'+regexmatch[0]+'/'
                    #print(csv_directory)
                
                
                
                    for csv in os.listdir(csv_directory):
                        #print('joined data directory:', csv_directory)
                                #loops though all csvs for each measurement (many)
                        #print(csv)
                        if 'Objects_Population' in csv:
                            #print(csv_list)
                            csv_object_reference = csv[21:]

                            csv_slideID=filename[:4].upper()
                            csvpath="".join([csv_directory, csv])
                                        #path = {'csv_path': csvpath}
                            csvdata={'csvpath': csvpath, 'csvname': csv_object_reference, 'slideId':csv_slideID}
                            csv_list.append(csvdata)

    
    
        #print('csvs here:', csv_list)
                #creates list of all csvpaths in both measurements x14
                
    for index, csv in enumerate(csv_list, start=0):
        
                    #assigns number to each list item
        #print('enumerated1')
        for index, image in enumerate(all_data, start=0):
            #print('enumerate 2: all data enumerated', index, image)
            for csvk, csvv in csv.items():
                
                for imagek, imagev in image.items():
                    
                    if csvv == imagev:
                        #print(csvv, 'is the same as', imagev)
                        newdict={**csv, **image}
                        joined_data.append(newdict)
    #print(joined_data)
                        
    return(joined_data)

def concat_data(analysis_file_list, directory):
    list2concat=[]
    
    for analysis_filename in analysis_file_list:
        df=pd.read_csv(directory+'/'+analysis_filename, delimiter='\t', engine='python')
        #if analysis type in csv name
        list2concat.append(df)
        slideId=analysis_filename[:4]
        # concatdata=slideId+'_'+analysis_name
    
    concatdata=pd.concat(list2concat, ignore_index=True)           
    return concatdata

def concatnormfiles(inputdir, outputdir, project, subproject):
    file_list=[]

# List of files in dir
    workingdir='/home/staffan/Code_KR/'+project+'_'+subproject+'/In_Progress/'
    print(workingdir)

    file_l=os.listdir(workingdir)
    for i in file_l:
        if '.csv' in i:
            file_list.append(i)
            
    regex = '(\_\d\d\_|\_\D\D\_|\_\D\D\D\_)([a-zA-Z0-9\_\+]*)'

    # Get everything after --_ or ++_ and before .txt
    # We use this to create ana_list, uses a regex to grab everything between those strings.
    analysis_list = [re.search(regex, filename).group(2) for filename in file_list]
    # Make the list only populated with unique items
    analysis_list = list(set(analysis_list))
    print('1', analysis_list)

    analysis_files_dict = {}
    for analysis_item in analysis_list:
        print('2', analysis_item, analysis_list)
        analysis_files_dict[analysis_item] = []
        for filename in file_list:
            if analysis_item in filename:
                print('3', filename)
                analysis_files_dict[analysis_item].append(filename)
                #print('4', analysis_files_dict)

    for analysis_item, analysis_filelist in analysis_files_dict.items():
        print('4', analysis_item)
        output = concat_data(analysis_filelist, workingdir)
        output=transform_px_to_um2 (output)
        fp=outputdir+project+'/'+subproject+'/ObjectResults/'+project+'_'+subproject+'_{}.csv'.format(analysis_item)
        
        print('5', analysis_item)
        output = output[~output.index.duplicated()]
        output.to_csv(fp, sep='\t', encoding='utf-8')
        print('6', fp)
                                                                                    
    return output, fp
    
def transform_px_to_um2 (output):
    
    for col in output.columns:
            if 'Number of Spots per Area' in col:
                output.loc[output[col] > 0, col] = (output[col]*44.7)
            if 'Region_Area' in col:
                output.loc[output[col] > 0, col] = round((output[col]/44.7), 2)
                
    return output
    
def setupfolders (project, subproject, workingdir, outputdir):
    outputfolderlist= ['CellCountResults', 'Figures', 'ObjectResults']
    
    for direc in outputfolderlist: 
        path=os.path.join(outputdir+project+'/'+subproject+'/'+direc)
        if os.path.exists(path):
            print('exists')
        else:
            os.mkdir(path)
    path2=os.path.join(workingdir+project+'_'+subproject+'/'+'In_Progress')
    if os.path.exists(path2):
            print('exists')
    else: 
        os.makedirs(path2)
           

In [46]:
###PARAMETERS
#datasets= dataset ids in omero
datasets=[876]
#workingdir= linux user home
workingdir='/home/staffan/Code_KR/'
outputdir='/mnt/Labox/Katherine/'
### outputdirectory== dir/project/subproject/subsub i.e. ddOPC/2/E1
### working == wdir/project_subproject i.e ddOPC_2
project='Sonic'
subproject='33'
user='Katherine'
password= 'Katherine'
host='localhost'
#subsubs=['E5']

In [47]:
#for subsub in subsubs:
harmonydir=outputdir+project+'/'+subproject+'/Evaluations/'

workingdir='/home/staffan/Code_KR/'
setupfolders(project, subproject, workingdir, outputdir)

jdata = harmony_omero_output(user, password, host, datasets, harmonydir)

for jdatadict in jdata:
    #print('rawdata is', jdatadict['csvpath'])
    csv, ROI_names, pol_area =segment_data (user, password, jdatadict['imagename'], jdatadict['imageId'], jdatadict['csvpath'], jdatadict['csvname'])

    for r, i, a in zip(csv, ROI_names, pol_area):
        csv_output,filepath=handle_csv (csv, ROI_names, jdatadict['imagename'], jdatadict['csvname'], pol_area, workingdir, project, subproject)
        csv_output.to_csv(filepath, sep='\t', encoding='utf-8')


objectresults, resultspath = concatnormfiles(workingdir, outputdir, project, subproject)
print('concat data went to:', resultspath)

connected
regex match Evaluation1
Getting metadata from OMERO...
1
1
1
1
1
data/OMERO/ManagedRepository/Katherine_152/2022-03/31/19-19-01.957/A4_F5P0T0.ome.tiff
Found the images
dataset is: BNIS json is
metapath found /mnt/Labox/Katherine/Sonic/33/Ref/BNIS__2022-03-25T09_38_18-Measurement 2/Images/FieldTables.json
0.5353217124938965
Reading images, identifying clusters (sections)...
BNIS
ref xml is: /mnt/Labox/Katherine/Sonic/33/Ref/BNIS__2022-03-25T09_38_18-Measurement 2/Images/Index.ref.xml
27.590454578399658
...Done
Converting ROI coordinates...
12.981391191482544
All Done!
0.0015206336975097656
0.21892905235290527
0.35628581047058105
Getting metadata from OMERO...
1
1
1
1
1
data/OMERO/ManagedRepository/Katherine_152/2022-03/31/19-17-20.108/A4_F4P0T0.ome.tiff
Found the images
dataset is: BNIS json is
metapath found /mnt/Labox/Katherine/Sonic/33/Ref/BNIS__2022-03-25T09_38_18-Measurement 2/Images/FieldTables.json
0.4887204170227051
Reading images, identifying clusters (sections)...
BN